## How to perform numerical integration in 2D + demo with Poisson and NB evidence

There are routines in scipy for 1D numerical intragtion, e.g., quad or trapz. While quad gets the actual function to integrated and just the interval for the integration and then performs the sampling and stuff itself, trapz only requires an array of precomputed function values on the domain in question and a stepsize or sampling points correspinding to the precomputed function values. It then uses the trapezoid method to calculate the integral. 

Now, for 2D there is a dblquad method that does just the same in higher dimensions. However, it gives negative values, probably due to rounding errors or so. So I might want to precalculate the function values and use something like trapz in 2D. However, this does not exist in numpy or scipy it seems. 

So let us check whether we can just use the 1D trapz method on every axis instead. Let us take a known 2D integral and check the different methods. 

In [ ]:
import numpy as np 
import scipy.integrate
import scipy.stats
import matplotlib.pyplot as plt 
import sys 
sys.path.append('../')
from utils import poisson_evidence, poisson_sum_evidence, nbinom_evidence, nbinom_sum_evidence
%matplotlib inline 

from scipy.special import gammaln, gamma, betaln, beta

import matplotlib as mpl
mpl.rcParams['axes.titlesize'] = 20
mpl.rcParams['axes.labelsize'] = 15
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['legend.fontsize'] = 15
mpl.rcParams['figure.figsize'] = (15, 5)

In [ ]:
# define the function 
fun = lambda x, y: x**2 + y**2

# define the domain 
x1 = -1 
x2 = 1 
y1 = -1 
y2 = 1

# set the result 
result = 8 / 3

In [ ]:
# construct grid of values for trapz 
x, y = np.meshgrid(np.linspace(-1, 1, 1000), np.linspace(-1, 1, 1000))
fun_mat = fun(x, y)

In [ ]:
# use quad 
rquad = scipy.integrate.dblquad(fun, -1, 1, lambda x: -1, lambda x: 1)

# use doubple trapz
rtrapz = np.trapz(np.trapz(fun_mat, np.linspace(-1, 1, 1000), axis=0), np.linspace(-1, 1, 1000), axis=0)

In [ ]:
rtrapz

In [ ]:
rquad

## Conclusion 
trapz is a bit less accurate, but it seems possible to just apply it along each axis of the precomputed matrix of function values. 

## Follow up: test the integration for complicated evidence integrals 

### First for Poisson evidence with ground truth

There are different ways for computing the Poisson evidence. One can just see a sample as a vector of i.i.d. single samples. Or one can see it in terms of the sufficient statistics, which is just the sum over the sample. The sum of i.i.d. Poisson samples is again Poissonian with a different rate. The corresponding derivations for the likelihood, prior and marginal likelihood differ by a multiplicative factor. I dont know which is the correct way to do it here, so I just try out both. I call the functions that take the point of view of the sufficient statistics **poisson_sum_evidence** and **poisson_sum_integrant**. 

The evidence function give the closed form solutions. The integrant function give the functions under the integral. We test the integration method by comparing the results. 

The different POV give different results unless corrected with multiplicative factor I derived. 

In [ ]:
def poisson_integrant(lam, x, k, theta, log=True): 
    sx = np.sum(x)   
    N = x.size
    prior = scipy.stats.gamma(a=k, scale=theta)
    # the prod of faculties 
    denominator = np.sum(gammaln(x + 1))
    
    # lam ** (sx) * np.exp(-N * lam) * prior.pdf(lam) / denominator
    log_integrant = sx * np.log(lam) - N * lam + np.log(prior.pdf(lam)) - denominator
    
    return np.exp(log_integrant)

def poisson_sum_integrant(eta, x, k, theta): 
    sx = np.sum(x)   
    N = x.size
    prior = scipy.stats.gamma(a=k, scale=N * theta)
    # the prod of faculties 
    denominator = gammaln(sx + 1)
    log_integrant = sx * np.log(eta) - eta + np.log(prior.pdf(eta)) - denominator
    
    return np.exp(log_integrant)

### Generate Poisson data 

In [ ]:
# set the prior 
k1 = 5. 
theta1 = 2. 
prior = scipy.stats.gamma(a=k1, scale=theta1)

# sample data 
sample_size = 100
x = scipy.stats.poisson.rvs(mu=prior.rvs(), size=sample_size)
sx = np.sum(x)
N = x.size
# calculate the scaling between the different POVs
i = sx * np.log(N) + np.sum(gammaln(x + 1)) - gammaln(sx + 1)
i = np.exp(i)

In [ ]:
x.shape

In [ ]:
# get the ground truths
result1 = poisson_sum_evidence(x, k1, theta1, log=False)
result2 = poisson_evidence(x, k1, theta1)
print('sx based: {} \n x based: {}'.format(result1, result2))
print('corrected: {}'.format(i * result2))

In [ ]:
# now do the integration, again for both POV separately
lambs = np.linspace(1e-8, 100, 1000)
values = np.array([poisson_integrant(lam, x, k1, theta1) for lam in lambs])
values2 = np.array([poisson_sum_integrant(N * lam, x, k1, theta1) for lam in lambs])

In [ ]:
print('traps x based, corrected: ', i * np.trapz(values, lambs))
print('traps sx based: ', np.trapz(values2, N * lambs))

In [ ]:
print('scipy x based: ', scipy.integrate.quad(poisson_integrant, 1e-8, 20, args=(x, k1, theta1))[0])

## Now test with NegBin evidence integral

There are different ways for formulating the evidence, e.g., for formulating the prefactor containing the binomial coef. I matched them to give the same result for the data POV. This points in the direction that this is the correct way. I also found a PhD thesis using that formulatin here: 
https://www.casact.org/pubs/forum/99wforum/wf99377.pdf

So, I should take this one for now. The sufficient stats POV has the additional problem that the integration and the closed form dont give the same result. 


In [ ]:
# first define the integrant functions in different POVs     
def nb_sum_integrant(p, r, x, a, b): 
    
    N = x.size
    sx = np.sum(x)
    bc = scipy.special.binom(sx + N * r - 1, sx)
    
    log_integrant = np.log(bc) + (b + N * r - 1) * np.log(1 - p) + (a + sx - 1) * np.log(p) - betaln(a, b)
    
    return np.exp(log_integrant)
    
def nb_evidence(r, x, a, b, log=True): 
    N = x.size
    sx = np.sum(x) 
    bc = scipy.special.binom(x + r - 1, x)
    
    log_evidence = np.sum(np.log(bc)) + betaln(a + sx, b + N * r) - betaln(a, b)
    
    return log_evidence if log else np.exp(log_evidence)
    
def nb_integrant(p, r, x, a, b): 
    
    N = x.size
    sx = np.sum(x)
    bc = scipy.special.binom(x + r - 1, x)
    
    fac = np.prod(bc) / beta(a, b)
    
    integrant = p**(a + sx - 1) * (1 - p)**(b + N*r - 1)
    
    return fac * integrant


def nb_integrant2(p, x, r, a, b): 
    N = x.size
    sx = np.sum(x)
    
    fac = np.sum(gammaln(x + r) - (gammaln(x + 1) + gammaln(r))) - betaln(a, b)
    integrant = np.log(p) * (a + sx - 1) + np.log(1 - p)*(b + N*r - 1)
    
    return np.exp(integrant + fac)

In [ ]:
# generate data 
r = 4
a = b = 2
prior = scipy.stats.beta(a=a, b=b)

sample_size = 100
x = scipy.stats.nbinom.rvs(n=r, p=prior.rvs(), size=sample_size)
x

In [ ]:
# calculate the integral values on a 1D grid 
ps = np.linspace(1e-7, .9999, 1000)
values1 = np.array([nb_sum_integrant(p, r, x, a, b) for p in ps])
values2 = np.array([nb_integrant(p, r, x, a, b) for p in ps])
values3 = np.array([nb_integrant2(p, x, r, a, b) for p in ps])

In [ ]:
# calculate ground truth: 
print('ana sx based: ', nbinom_sum_evidence(x, r, a, b, log=False))
print('ana x based: ', nb_evidence(r, x, a, b, log=False))
print('ana x new: ', nbinom_evidence(x, r, a, b, log=False))

In [ ]:
N = x.size
sx = np.sum(x)

print('traps sx based: ', np.trapz(values1, ps))
print('traps x based: ', np.trapz(values2, ps))
print('traps x new: ', np.trapz(values3, ps))